In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import pandas as pd

In [16]:
babies["x"].mean() - babies["y"].mean()

-2.4499999999999993

In [6]:
babies = pd.read_csv("babies.csv")

prior_α = 0.001
prior_β = 0.001

with pm.Model() as m:
    α1 = pm.Gamma("μ1", prior_α, prior_β)
    α2 = pm.Gamma("μ2", prior_α, prior_β)
    β1 = pm.Gamma("β1", prior_α, prior_β)
    β2 = pm.Gamma("β2", prior_α, prior_β)

    x = pm.Gamma("x", α1, β1, observed=babies["x"].to_numpy())
    y = pm.Gamma("y", α2, β2, observed=babies["y"].to_numpy())

    mean_diff = pm.Deterministic("mean_diff", α1 / β1 - α2 / β2)

    trace = pm.sample(
        10000,
        chains=4,
        tune=2000,
        cores=4,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )

    print(az.summary(trace))


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [μ1, μ2, β1, β2]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 2_000 tune and 10_000 draw iterations (8_000 + 40_000 draws total) took 32 seconds.
There were 26 divergences after tuning. Increase `target_accept` or reparameterize.
There were 18 divergences afte

             mean      sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
μ1         34.779  12.927  12.806   59.050      0.139    0.110    9341.0   
μ2         27.802  10.055  10.748   46.825      0.096    0.070   10975.0   
β1          3.607   1.351   1.323    6.153      0.014    0.012    9361.0   
β2          2.299   0.838   0.869    3.873      0.008    0.006   11008.0   
mean_diff  -2.460   0.756  -3.877   -1.018      0.004    0.003   40221.0   

           ess_tail  r_hat  
μ1           9453.0    1.0  
μ2          11971.0    1.0  
β1           9373.0    1.0  
β2          11829.0    1.0  
mean_diff   29189.0    1.0  


In [7]:
az.summary(trace, hdi_prob=.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
μ1,34.779,12.927,11.937,60.231,0.139,0.110,9341.0,9453.0,1.0
μ2,27.802,10.055,9.837,47.485,0.096,0.070,10975.0,11971.0,1.0
β1,3.607,1.351,1.251,6.288,0.014,0.012,9361.0,9373.0,1.0
β2,2.299,0.838,0.874,4.007,0.008,0.006,11008.0,11829.0,1.0
mean_diff,-2.460,0.756,-3.989,-0.997,0.004,0.003,40221.0,29189.0,1.0


In [18]:
wolves

,arctic,gender,x3,x7
0,0,0,5.55,2.00
1,0,0,5.94,2.07
2,0,0,5.98,1.94
3,0,0,5.55,1.90
4,0,0,5.63,1.93
5,0,0,5.63,1.83
6,0,1,5.16,1.79
7,0,1,5.12,1.75
8,0,1,4.92,1.63
9,1,0,5.28,1.60


In [18]:
wolves_females_only = wolves.query("gender == 1")
wolves_females_only

,arctic,gender,x3,x7
6,0,1,5.16,1.79
7,0,1,5.12,1.75
8,0,1,4.92,1.63
19,1,1,5.28,1.77
20,1,1,5.24,1.78
21,1,1,5.47,1.85
22,1,1,5.24,1.74
23,1,1,5.75,2.01
24,1,1,5.39,1.77


In [26]:
wolves

,arctic,gender,x3,x7
0,0,0,5.55,2.00
1,0,0,5.94,2.07
2,0,0,5.98,1.94
3,0,0,5.55,1.90
4,0,0,5.63,1.93
5,0,0,5.63,1.83
6,0,1,5.16,1.79
7,0,1,5.12,1.75
8,0,1,4.92,1.63
9,1,0,5.28,1.60


In [51]:
from pymc.math import dot

wolves = pd.read_csv("wolves.csv")

X = wolves.iloc[:, 1:].to_numpy()
# add intercept column to X
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
y = wolves["arctic"].to_numpy()

τ = .01 # precision
μ = 0

new_data = np.array([1, 1, 5.28, 1.78], ndmin=2)

with pm.Model() as m:
    X_data = pm.Data("X_data", X_aug, mutable=True)
    y_data = pm.Data("y_data", y, mutable=False)

    betas = pm.Normal("beta", mu=μ, tau=τ, shape=X_aug.shape[1])

    p = dot(X_data, betas)

    lik = pm.Bernoulli("y", logit_p=p, observed=y_data)

    trace = pm.sample(
        5000,
        chains=4,
        tune=2000,
        cores=4,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )
    
    print(az.summary(trace, hdi_prob=.95))
    
    # prediction
    pm.set_data({"X_data": new_data})
    post_pred = pm.sample_posterior_predictive(trace)
    
    print(az.summary(post_pred, hdi_prob=.95))

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 2_000 tune and 5_000 draw iterations (8_000 + 20_000 draws total) took 32 seconds.
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Coul

           mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
beta[0]  -3.774  7.748   -19.014     11.048      0.106    0.075    5300.0   
beta[1]   1.034  1.114    -1.146      3.206      0.015    0.010    5636.0   
beta[2]   4.534  2.311     0.103      9.042      0.038    0.027    3746.0   
beta[3] -11.412  5.416   -22.653     -1.580      0.083    0.059    4306.0   

         ess_tail  r_hat  
beta[0]    7687.0    1.0  
beta[1]    7961.0    1.0  
beta[2]    5675.0    1.0  
beta[3]    6270.0    1.0  


      mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
y[0]  0.69  0.462       0.0        1.0      0.003    0.002   19180.0   

      ess_tail  r_hat  
y[0]   19180.0    1.0  


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Could not extract data from symbolic observation y
  warnings.warn(f"Could not extract data from symbolic observation {obs}")
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/arviz/stats/stats.py:1180: UserWarning: Selecting first found group: posterior_predictive
  warnings.warn(f"Selecting first found group: {data.groups()[0]}")


In [39]:
X_aug.shape

(25, 4)

In [47]:
new_data = np.array([1, 1, 5.28, 1.78], ndmin=2)

with m:  
    pm.set_data({"X_data": new_data})
    post_pred = pm.sample_posterior_predictive(trace)
    
    print(az.summary(post_pred, hdi_prob=.95))

       mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
y[0]  0.685  0.465       0.0        1.0      0.003    0.002   18997.0   

      ess_tail  r_hat  
y[0]   18997.0    1.0  


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Could not extract data from symbolic observation y
  warnings.warn(f"Could not extract data from symbolic observation {obs}")
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/arviz/stats/stats.py:1180: UserWarning: Selecting first found group: posterior_predictive
  warnings.warn(f"Selecting first found group: {data.groups()[0]}")


In [46]:
az.summary(trace2, hdi_prob=.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],0.379,5.266,-9.796,10.649,0.054,0.038,9468.0,12599.0,1.0
beta[1],0.324,0.608,-0.878,1.504,0.006,0.004,9688.0,11926.0,1.0
beta[2],0.139,1.381,-2.595,2.835,0.016,0.011,7360.0,9228.0,1.0
beta[3],0.546,2.720,-4.719,5.968,0.028,0.020,9581.0,12159.0,1.0
μ[0],2.243,0.596,1.099,3.439,0.006,0.004,9529.0,13531.0,1.0
μ[1],2.336,0.504,1.355,3.327,0.004,0.003,18603.0,21855.0,1.0
μ[2],2.270,0.442,1.417,3.148,0.004,0.003,15567.0,21216.0,1.0
μ[3],2.189,0.389,1.414,2.940,0.004,0.003,11800.0,18988.0,1.0
μ[4],2.216,0.380,1.459,2.950,0.003,0.002,14020.0,21667.0,1.0
μ[5],2.162,0.298,1.590,2.754,0.002,0.001,27100.0,28914.0,1.0


In [47]:
new_data = np.array([[1], [5.28], [1.78]])
new_data

array([[1.  ],
       [5.28],
       [1.78]])

In [48]:
with m:
    pm.set_data({"data": new_data})
    post_pred = pm.sample_posterior_predictive(trace2, var_names=["μ"])

In [49]:
post_pred["μ"]

array([[2.91022786, 2.45416881, 2.19590352, ..., 2.38694777, 2.06569752,
        2.21628415],
       [1.23532782, 1.85097573, 2.08396105, ..., 2.43898403, 3.0231088 ,
        2.67196215],
       [2.95457112, 2.95084389, 2.0861793 , ..., 1.74300785, 2.86250852,
        1.76096007],
       ...,
       [2.11159805, 2.63886368, 2.74800936, ..., 2.53324051, 3.15110295,
        2.73479983],
       [1.96518856, 2.5386851 , 2.71925125, ..., 2.48004718, 3.07162063,
        2.69789203],
       [1.68504322, 1.84937135, 2.27027067, ..., 2.69247844, 2.38173276,
        2.74661887]])

In [50]:
az.summary(post_pred)

/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/arviz/data/base.py:216: UserWarning: More chains (40000) than draws (25). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
μ,2.261,0.462,1.39,3.143,0.001,0.001,243639.0,376603.0,1.13


In [3]:
mn = pd.read_csv("micronuclei.csv")

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [4., 5.],
       [4., 5.],
       [4., 6.]])

In [58]:
from pymc.math import exp

mn = pd.read_csv("micronuclei.csv")

new_data = np.array([3.5])

with pm.Model() as m:
    x_data = pm.Data("x_data", mn["x"].to_numpy())
    y_data = pm.Data("y_data", mn["y"].to_numpy())
    
    beta0 = pm.Normal("beta0", mu=0, tau=.0001)
    beta1 = pm.Normal("beta1", mu=0, tau=.0001)
    
    λ = exp(beta0 + beta1 * x_data)
            
    y = pm.Poisson("y", mu=λ, observed=y_data)
    
    trace = pm.sample(
        3000, # keeping number of samples consistent, but this is not an exact
        chains=4,
        tune=1000,
        cores=4,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )
    
    trace_burned = trace.sel(draw=slice(500, None)).posterior.reset_index(["chain", "draw"])

    az.summary(trace_burned, hdi_prob=.95)
    
    pm.set_data({"x_data": new_data})
    ppc = pm.sample_posterior_predictive(trace_burned)
    print(az.summary(ppc, hdi_prob=.95))


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/data.py:647: FutureWarning: The `mutable` kwarg was not specified. Currently it defaults to `pm.Data(mutable=True)`, which is equivalent to using `pm.MutableData()`. In v4.1.0 the default will change to `pm.Data(mutable=False)`, equivalent to `pm.ConstantData`. Set `pm.Data(..., mutable=False/True)`, or use `pm.ConstantData`/`pm.MutableData`.
  warnings.warn(
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0, beta1]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 12 seconds.
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Coul

       mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
y[0]  0.625  0.792       0.0        2.0      0.008    0.006   10132.0   

      ess_tail  r_hat  
y[0]    9598.0    1.0  


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Could not extract data from symbolic observation y
  warnings.warn(f"Could not extract data from symbolic observation {obs}")
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/arviz/stats/stats.py:1180: UserWarning: Selecting first found group: posterior_predictive
  warnings.warn(f"Selecting first found group: {data.groups()[0]}")


In [59]:
ppc.

Inference data with groups:
	> posterior_predictive

In [29]:
trace = trace_burned.posterior.reset_index(["chain", "draw"])
with m:
    ppc1 = pm.fast_sample_posterior_predictive(trace)

In [30]:
az.summary(ppc1)

/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/arviz/data/base.py:216: UserWarning: More chains (18000) than draws (6000). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
y,0.303,0.621,0.0,1.0,0.002,0.001,146026.0,149346.0,1.07


In [36]:
trace_burned = trace3.sel(draw=slice(500, None)).posterior.reset_index(["chain", "draw"])

In [6]:
az.summary(trace_burned, hdi_prob=.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0,-2.815,0.064,-2.941,-2.688,0.001,0.001,4065.0,4161.0,1.0
beta1,0.671,0.020,0.631,0.709,0.000,0.000,4012.0,4067.0,1.0


In [6]:
az.summary(ppc1, hdi_prob=.95)

/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/arviz/data/base.py:216: UserWarning: More chains (20000) than draws (6000). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
y,0.303,0.621,0.0,2.0,0.001,0.001,162229.0,165926.0,1.07


In [37]:
new_data = np.full(6000, 3.5)
with m:
    pm.set_data({"x_data": new_data})
    ppc = pm.fast_sample_posterior_predictive(trace_burned)
az.summary(ppc, hdi_prob=.95)

/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/arviz/data/base.py:216: UserWarning: More chains (18000) than draws (6000). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
y,0.628,0.792,0.0,2.0,0.0,0.0,107965766.0,108025485.0,1.0


https://lucianopaz.github.io/2019/08/19/pymc3-shape-handling/

https://benslack19.github.io/data%20science/statistics/pymc-linreg_entry04/  
better posterior predictive

In [2]:
data = pd.read_csv("micronuclei.csv")
X = data['x'].values
y = data['y'].values

with pm.Model() as m:
    # priors
    beta_0 = pm.Normal("beta_0", mu=0, sigma=1000)
    beta_dose = pm.Normal("beta_dose", mu=0, sigma=1000)
    
    lambda_ = pm.Deterministic("lambda", pm.math.exp(beta_0 + beta_dose * X))

    # likelihood
    y_pred = pm.Poisson("y_pred", lambda_, observed=y)

    # start sampling
    trace = pm.sample(
        3000,  # samples
        chains=4,
        tune=1000,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )
    
az.summary(trace, hdi_prob=0.95)


    


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_dose]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 13 seconds.
The acceptance probability does not match the target. It is 0.6532, but should be close to 0.8. Try to increase the

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,-2.812,0.066,-2.944,-2.688,0.001,0.001,2432.0,2458.0,1.0
beta_dose,0.670,0.020,0.631,0.710,0.000,0.000,2490.0,2643.0,1.0
lambda[0],0.060,0.004,0.052,0.068,0.000,0.000,2432.0,2458.0,1.0
lambda[1],0.060,0.004,0.052,0.068,0.000,0.000,2432.0,2458.0,1.0
lambda[2],0.060,0.004,0.052,0.068,0.000,0.000,2432.0,2458.0,1.0
...,...,...,...,...,...,...,...,...,...
lambda[5995],0.878,0.026,0.827,0.930,0.000,0.000,5779.0,7475.0,1.0
lambda[5996],0.878,0.026,0.827,0.930,0.000,0.000,5779.0,7475.0,1.0
lambda[5997],0.878,0.026,0.827,0.930,0.000,0.000,5779.0,7475.0,1.0
lambda[5998],0.878,0.026,0.827,0.930,0.000,0.000,5779.0,7475.0,1.0


In [3]:
%%timeit -c
with m:
    test = pm.fast_sample_posterior_predictive(trace)


AttributeError: module 'pymc' has no attribute 'fast_sample_posterior_predictive'

In [5]:
%%timeit -c
with m:
    test2 = pm.sample_posterior_predictive(trace)


6.7 s ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
trace

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> observed_data

In [12]:
print(trace.posterior)

<xarray.Dataset>
Dimensions:       (chain: 4, draw: 3000, lambda_dim_0: 6000)
Coordinates:
  * chain         (chain) int64 0 1 2 3
  * draw          (draw) int64 0 1 2 3 4 5 6 ... 2994 2995 2996 2997 2998 2999
  * lambda_dim_0  (lambda_dim_0) int64 0 1 2 3 4 5 ... 5995 5996 5997 5998 5999
Data variables:
    beta_0        (chain, draw) float64 -2.587 -2.612 -2.618 ... -2.904 -2.936
    beta_dose     (chain, draw) float64 0.6032 0.5952 0.6069 ... 0.6954 0.6921
    lambda        (chain, draw, lambda_dim_0) float64 0.07526 0.07526 ... 0.8457
Attributes:
    created_at:                 2022-03-21T12:38:03.855081
    arviz_version:              0.11.4
    inference_library:          pymc
    inference_library_version:  4.0.0b4
    sampling_time:              13.04793906211853
    tuning_steps:               1000


In [60]:
# q3 full
import pymc as pm
import arviz as az
import pandas as pd
import numpy as np
from pymc.math import exp


mn = pd.read_csv("micronuclei.csv")

# for prediction in part b
new_data = np.array([3.5])

with pm.Model() as m:
    # mutable=True allows us to use pm.set_data() later
    x_data = pm.Data("x_data", mn["x"].to_numpy(), mutable=True)
    y_data = pm.Data("y_data", mn["y"].to_numpy(), mutable=False)

    beta0 = pm.Normal("beta0", mu=0, tau=0.0001)
    beta1 = pm.Normal("beta1", mu=0, tau=0.0001)

    λ = exp(beta0 + beta1 * x_data)

    y = pm.Poisson("y", mu=λ, observed=y_data)

    # keeping number of samples consistent with problem statement
    # but note that underlying algorithm is different, so these 3000
    # are getting a lot more done than BUGS.
    trace = pm.sample(
        3000,
        chains=4,
        tune=1000,  # Not the same as burning samples!
        cores=4,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )

    # burning first 500 samples as per problem description,
    # then resetting index so it will still work with sample_posterior_predictive
    trace_burned = trace.sel(draw=slice(500, None)).posterior.reset_index(
        ["chain", "draw"]
    )

    print(az.summary(trace_burned, hdi_prob=0.95))

    # prediction
    pm.set_data({"x_data": new_data})
    ppc = pm.sample_posterior_predictive(trace_burned)
    print(az.summary(ppc, hdi_prob=0.95))

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0, beta1]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 14 seconds.
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Coul

        mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
beta0 -2.812  0.064    -2.936     -2.688      0.001    0.001    2166.0   
beta1  0.670  0.020     0.632      0.708      0.000    0.000    2156.0   

       ess_tail  r_hat  
beta0    2451.0    1.0  
beta1    2493.0    1.0  


       mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
y[0]  0.631  0.795       0.0        2.0      0.008    0.006   10406.0   

      ess_tail  r_hat  
y[0]    9713.0    1.0  


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Could not extract data from symbolic observation y
  warnings.warn(f"Could not extract data from symbolic observation {obs}")
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/arviz/stats/stats.py:1180: UserWarning: Selecting first found group: posterior_predictive
  warnings.warn(f"Selecting first found group: {data.groups()[0]}")


In [64]:
np.mean(trace_burned.beta0)

<xarray.DataArray 'beta0' ()>
array(-2.81221892)

In [124]:
from scipy.stats import poisson

beta0 = float(np.mean(trace_burned.beta0))
beta1 = float(np.mean(trace_burned.beta1))

pred_lam = np.exp(beta0 + beta1 * 3.5)
pred = poisson.mean(mu=pred_lam)

print(f"predicted value at x=3.5 is {pred}")

predicted value at x=3.5 is 0.6275231028050723


In [76]:
float(pred_lam)

0.6275231028050723

In [96]:
beta1

0.6703554658218572

In [126]:
# q2 full
import pandas as pd
import arviz as az
import pymc as pm
import numpy as np
from pymc.math import dot



wolves = pd.read_csv("wolves.csv")

X = wolves.iloc[:, 1:].to_numpy()

# add intercept column to X by prepending a column of 1s
# this allows us to
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
y = wolves["arctic"].to_numpy()

τ = 0.01  # precision
μ = 0

# values for prediction
# note that first column must be 1 for intercept. it goes [intercept, sex, x3, x7]
# also, array must have 2 dimensions like the X_aug array
new_data = np.array([1, 1, 5.28, 1.78], ndmin=2)

with pm.Model() as m:
    # mutable=True allows us to use pm.set_data() later
    X_data = pm.Data("X_data", X_aug, mutable=True)
    y_data = pm.Data("y_data", y, mutable=False)

    betas = pm.Normal("beta", mu=μ, tau=τ, shape=X_aug.shape[1])

    p = dot(X_data, betas)

    lik = pm.Bernoulli("y", logit_p=p, observed=y_data)

    trace = pm.sample(
        10000,
        chains=4,
        tune=2000,
        cores=4,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )

    print(az.summary(trace, hdi_prob=0.95)) 

    # prediction
    pm.set_data({"X_data": new_data})
    post_pred = pm.sample_posterior_predictive(trace)

    print(az.summary(post_pred, hdi_prob=0.95))

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 2_000 tune and 10_000 draw iterations (8_000 + 40_000 draws total) took 48 seconds.
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Cou

           mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
beta[0]  -3.753  7.769   -18.999     11.259      0.076    0.053   10598.0   
beta[1]   1.028  1.108    -1.135      3.198      0.010    0.007   11543.0   
beta[2]   4.526  2.339     0.139      9.246      0.027    0.019    7382.0   
beta[3] -11.400  5.457   -22.527     -1.185      0.059    0.042    8525.0   

         ess_tail  r_hat  
beta[0]   14920.0    1.0  
beta[1]   15955.0    1.0  
beta[2]   11513.0    1.0  
beta[3]   11408.0    1.0  


       mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
y[0]  0.687  0.464       0.0        1.0      0.002    0.002   36742.0   

      ess_tail  r_hat  
y[0]   36742.0    1.0  


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Could not extract data from symbolic observation y
  warnings.warn(f"Could not extract data from symbolic observation {obs}")
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/arviz/stats/stats.py:1180: UserWarning: Selecting first found group: posterior_predictive
  warnings.warn(f"Selecting first found group: {data.groups()[0]}")


In [128]:
trace

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> constant_data

In [154]:
from scipy.stats import bernoulli


beta0 = float(np.mean(trace.posterior.beta[:, :, 0]))
beta1 = float(np.mean(trace.posterior.beta[:, :, 1]))
beta2 = float(np.mean(trace.posterior.beta[:, :, 2]))
beta3 = float(np.mean(trace.posterior.beta[:, :, 3]))


p = beta0 + beta1 * 1 + beta2 * 5.28 + beta3 * 1.78
logit_p = np.log(p/(1-p))
p2 = np.exp(p)/(np.exp(p) + 1)
pred = bernoulli.mean(p=p2)

print(f"predicted value at x=3.5 is {pred}")

predicted value at x=3.5 is 0.7073226106933465


In [153]:
p2

0.7073226106933465

In [5]:
#q3 full 
import pymc as pm
import arviz as az
import pandas as pd
import numpy as np
from pymc.math import exp



mn = pd.read_csv("micronuclei.csv")

# for prediction in part b
new_data = np.array([3.5])

with pm.Model() as m:
    # mutable=True allows us to use pm.set_data() later
    x_data = pm.Data("x_data", mn["x"].to_numpy(), mutable=True)
    y_data = pm.Data("y_data", mn["y"].to_numpy(), mutable=False)

    beta0 = pm.Normal("beta0", mu=0, tau=0.0001)
    beta1 = pm.Normal("beta1", mu=0, tau=0.0001)

    λ = exp(beta0 + beta1 * x_data)

    y = pm.Poisson("y", mu=λ, observed=y_data)

    # keeping number of samples consistent with problem statement
    # but note that underlying algorithm is different, so these 3000
    # are getting a lot more done than BUGS.
    trace = pm.sample(
        3000,
        chains=4,
        tune=1000,  # Not the same as burning samples!
        cores=4,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )

    # burning first 500 samples as per problem description,
    # then resetting index so it will still work with sample_posterior_predictive
    trace_burned = trace.sel(draw=slice(500, None)).posterior.reset_index(
        ["chain", "draw"]
    )

    print(az.summary(trace_burned, hdi_prob=0.95))

    # prediction
    pm.set_data({"x_data": new_data})
    ppc = pm.sample_posterior_predictive(trace_burned)
    print(az.summary(ppc, hdi_prob=0.95))

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0, beta1]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 14 seconds.
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Coul

        mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
beta0 -2.812  0.064    -2.936     -2.688      0.001    0.001    2166.0   
beta1  0.670  0.020     0.632      0.708      0.000    0.000    2156.0   

       ess_tail  r_hat  
beta0    2451.0    1.0  
beta1    2493.0    1.0  


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Could not extract data from symbolic observation y
  warnings.warn(f"Could not extract data from symbolic observation {obs}")
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/arviz/stats/stats.py:1180: UserWarning: Selecting first found group: posterior_predictive
  warnings.warn(f"Selecting first found group: {data.groups()[0]}")


       mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
y[0]  0.621  0.784       0.0        2.0      0.008    0.006   10110.0   

      ess_tail  r_hat  
y[0]   10002.0    1.0  


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0, beta1]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 16 seconds.
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Coul

        mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
beta0 -2.812  0.064    -2.936     -2.688      0.001    0.001    2166.0   
beta1  0.670  0.020     0.632      0.708      0.000    0.000    2156.0   

       ess_tail  r_hat  
beta0    2451.0    1.0  
beta1    2493.0    1.0  


       mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
y[0]  0.637  0.801       0.0        2.0      0.008    0.006    9790.0   

      ess_tail  r_hat  
y[0]    9830.0    1.0  
6.73 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Could not extract data from symbolic observation y
  warnings.warn(f"Could not extract data from symbolic observation {obs}")
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/arviz/stats/stats.py:1180: UserWarning: Selecting first found group: posterior_predictive
  warnings.warn(f"Selecting first found group: {data.groups()[0]}")
